In [ ]:
def hour_to_date(dt):
    return dt.date()

def round_to_hour(dt):
    dt_start_of_hour = dt.replace(minute=0, second=0, microsecond=0)
    return dt_start_of_hour


def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import numpy as np
    import pandas as pd
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    from itertools import repeat
    import random,pickle
#     import plotly
    with open('/home/jupyter/sneupane/MOODS/pickled_files/colors.pickle', 'rb') as handle:
        colors = pickle.load(handle)
    
    with open('../pickled_files/user_days_week.pickle', 'rb') as handle:
        user_days_week = pickle.load(handle)
    with open('../pickled_files/final_df_splitted.pickle', 'rb') as handle:
        final_df_splitted = pickle.load(handle)
    user_df=final_df_splitted.loc[final_df_splitted.user==user_id]
    user_df["start_hour"]=user_df["starttime"].apply(round_to_hour).dt.hour

    user_df["end_hour"]=user_df["endtime"].apply(round_to_hour).dt.hour
    
    random.shuffle(colors)
    if len(user_df)>0:
        colors=['#ff9868', '#ff7c51', '#ff5c37', '#ff2b16', '#ec0000', '#d00000', '#b50000', '#9b0000', '#820000', '#690000']

        user_df_hour_group=user_df.groupby(["counter","start_hour"],as_index=False)["value"].mean()
        max_counter=max(user_df_hour_group["counter"])

        day_hour_count={}
        hour_window=[]
        
        for hour in range(0,24):
            hour_window.append(str(hour))
        hour_window.reverse()
        day_hour_count={}
        for hour in range(23,0,-1):
            temp_hour=[None for i in range(1,max_counter+1)]
            user_df_group_hour=user_df_hour_group.loc[user_df_hour_group["start_hour"]==hour]

            if len(user_df_group_hour)>0:
                counter_hour_count=dict(zip(user_df_group_hour.counter, user_df_group_hour.value))

                for counter in range(1,max_counter+1):

                    if counter in counter_hour_count:
                        temp_hour[counter-1]=counter_hour_count[counter]
                    else:
                        temp_hour[counter-1]=None
                day_hour_count[hour]=temp_hour
            else:
                day_hour_count[hour]=temp_hour

        count_day_user=user_days_week[user_id][0]["Count_days"]
        week_day_user=user_days_week[user_id][0]["Weeks_days"]
        last_date=list(count_day_user.keys())[list(count_day_user.values()).index(max_counter)]
#         print(last_date)
        last_week=week_day_user[last_date]
#         print(last_week)
        counter_day_dict={}
        days=[]
        for count in range(1,max_counter+1):
            counter_day_dict[count]=list(count_day_user.keys())[list(count_day_user.values()).index(count)]
            days.append(str(list(count_day_user.keys())[list(count_day_user.values()).index(count)]))

        week_vals=[]
        for week in range(1,last_week+1):
            week_vals.append(list(week_day_user.keys())[list(week_day_user.values()).index(week)])
     
        fig = go.Figure(data=go.Heatmap(hoverongaps = False,
        z=list(day_hour_count.values()),
        x=days, y=hour_window,
#         y=[i for i in range(23,0,-1)],
        colorscale=colors,zmin=0, zmax=1, hovertemplate=
                                  
                               "Day # : %{x}<br>"+
                                "Hour of day #: %{y}<br>" +
                                "Stress Probability : %{z}"
                                "<extra></extra>",
                   colorbar = dict(thickness=7,tick0=0,title="", dtick=1)),)
        fig.update_yaxes(nticks=48)
        
        fig.update_layout(
                title='',
                    xaxis_nticks=100,xaxis_dtick=1,yaxis_dtick=1,yaxis_title="Hour of the day", )
        fig.update_xaxes(showticklabels=True,showgrid=True ,type="category",tickangle=90,tickvals=week_vals)
        fig.update_yaxes(
            side="top",showgrid=True ,
                          ticks="outside",mirror=True,
#                           tickvals=y,
                  type="category",
            showline=True,linewidth=1,
            
        )
        fig.update_xaxes(side="bottom",ticks="outside",mirror=True,
                  showline=True,linecolor="Black",
            tickson="labels",
            ticklen=5)
        fig.update_layout(template="plotly_white")
#         fig.layout.xaxis.color = 'Black'
#         fig.layout.yaxis.color = 'gray'
#         fig.update_xaxes(fixedrange=True)
#         fig.update_yaxes(fixedrange=True)
#         fig.layout.xaxis.gridcolor="black"
#         fig.layout.xaxis.gridwidth=0.5
#         fig.layout.xaxis.tickfont={"family":"Times New Roman","size":15}
#         fig.layout.yaxis.gridcolor="black"
#         fig.layout.yaxis.gridwidth=0.5
        fig.update_layout(hoverlabel=dict(
                                            bgcolor="white",
                                            font_size=16,
                                            font_family="Rockwell",
                                            bordercolor="Black"
                                        )
                                    )
        fig.update_layout(
                    autosize=True,

                    margin=dict(
                       l=60,
                            r=30,
                            b=40,
                            t=40,

                    ),)
        
        
        data_summary_user=data_summary_user_current["HeatMap_day_count"]
      

       
       
        
        
        return "HeatMap_day_count",fig,data_summary_user
    else:
        return "","",""